<a href="https://colab.research.google.com/github/afaq5587/Lang-chain/blob/main/langchain_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Assalam o Alaikum to our best teachers.

I have developed a LangChain
agent that integrates three powerful tools: OpenAI, the Tavily search engine, and the OpenWeatherMap model. This agent is designed to provide smooth and accurate answers to a wide range of queries. Feel free to ask it anything, and it will respond effortlessly.

In [1]:
!pip install langchain langchain_community langchain_google_genai pyowm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [7]:
from IPython import get_ipython
from IPython.display import display
from google.colab import userdata
import os
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain_community.tools import TavilySearchResults
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType,load_tools
from langchain_community.utilities.openweathermap import OpenWeatherMapAPIWrapper
from typing_extensions import Final
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["OPENWEATHERMAP_API_KEY"] = userdata.get("OPENWEATHERMAP_API_KEY")
OPENWEATHERMAP_API_KEY = os.environ.get("OPENWEATHERMAP_API_KEY")
open_api = OpenWeatherMapAPIWrapper(openweathermap_api_key=OPENWEATHERMAP_API_KEY)
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")

search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)
@tool
def custom_search(query:str) :
    """pick the user query and search it in search engine

    """
    return search.invoke({"query": query})

llm=ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model='gemini-1.5-flash',verbose=True)
llm_with_tools = llm.bind_tools([custom_search])
def tavily_search(query:str):
    return custom_search(query)

weather_tool = Tool(
    name="OpenWeatherMap",
    func=open_api.run,
    description="Useful for when you need to get weather information for a location."
)
search_tool = Tool(
    name="Search",
    func=tavily_search,
    description="Useful for when you need to answer questions about current events. You should ask targeted questions."
)
tools = [search_tool,weather_tool ]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


In [10]:
user_query = input("Enter your query: ")
while user_query != "quit":
    response = agent.run(user_query)
    print(response)
    user_query = input("Enter your query: ")

Enter your query: what is 56 * 89


> Entering new AgentExecutor chain...
Thought: I can calculate this directly without needing external tools.
Thought: I now know the final answer
Final Answer: 4984

> Finished chain.
4984
Enter your query: what is python


> Entering new AgentExecutor chain...
Thought: I need to find information about Python.  A web search will be the best way to get a concise and accurate definition.
Action: Search
Action Input: "What is Python?"
Observation: [{'url': 'https://www.geeksforgeeks.org/python-programming-language-tutorial/', 'content': 'Python is a versatile and beginner-friendly programming language widely used in web development, data science, artificial intelligence, and automation, known for its simplicity, extensive libraries, and strong community support.'}, {'url': 'https://www.pythontutorial.net/getting-started/what-is-python/', 'content': 'Learn what Python is, why you should learn it, and how it differs from other programming languages. Pytho